In [48]:
import sys, os
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
from pandas import DataFrame
import time
from openpyxl.workbook import Workbook

In [52]:
sleep_sec = 0.5

wb = Workbook()

# User-Agent를 입력해주세요.
headers = {'User-Agent' : '________________'}

query = '데이터'
yesterday = (datetime.today() - timedelta(1)).strftime("%Y.%m.%d")

def news_crawling():
    
    service = Service(executable_path=ChromeDriverManager().install())
    browser = webdriver.Chrome(service=service)


    print('브라우저를 실행시킵니다(자동 제어)\n')

    news_url = 'https://search.naver.com/search.naver?where=news&query={0}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={1}&de={1}]'.format(query, yesterday)
    browser.get(news_url)
    browser.maximize_window()
    time.sleep(sleep_sec)
    

    print('\n크롤링을 시작합니다.')

    #####동적 제어로 페이지 넘어가며 크롤링
    news_dict = {}
    idx = 1
    cur_page = 1
    news_num = 1000000

    while True:

        table = browser.find_element(By.XPATH, '//ul[@class="list_news"]')
        li_list = table.find_elements(By.XPATH, './li[contains(@id, "sp_nws")]')
        area_list = [li.find_elements(By.XPATH,'.//div[@class="news_wrap api_ani_send"]') for li in li_list]
                
        for a in area_list[:min(len(area_list), news_num-idx+1)]:
            n = a.find_element(By.XPATH,'.//a[@class="news_tit"]')
            n_url = n.get_attribute('href')
                    
            try:
                img = a.find_element(By.CSS_SELECTOR,'a.dsc_thumb ').find_element(By.CSS_SELECTOR, 'img')
                img = img.get_attribute('src')
                
            except:
                img = " "
                    
            news_dict[idx] = {'Title' : n.get_attribute('title'),
                            'url' : n_url,
                            'thumbnail': img}
            
            idx += 1
            
            
        try:
            next_btn = browser.find_element(By.CSS_SELECTOR, 'a.btn_next')
            next_btn.click()
        
            cur_page +=1

            pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
            next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

            browser.get(next_page_url)
            time.sleep(sleep_sec)
            
        except:
            print('\n브라우저를 종료합니다.\n' + '=' * 100)
            time.sleep(0.7)
            browser.close()
            break
        
    print('데이터프레임 변환\n')
    
    news_df = DataFrame(news_dict).T

    folder_path = os.getcwd()
    xlsx_file_name = '{}_{}.xlsx'.format(query, yesterday)

    news_df.to_excel(xlsx_file_name, index=False)

    print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))
    
news_crawling()

브라우저를 실행시킵니다(자동 제어)


크롤링을 시작합니다.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//ul[@class="list_news"]"}
  (Session info: chrome=123.0.6312.58); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x6050219fb993 <unknown>
#1 0x6050216f6136 <unknown>
#2 0x605021740d48 <unknown>
#3 0x605021740e01 <unknown>
#4 0x605021783e44 <unknown>
#5 0x605021762cfd <unknown>
#6 0x605021781319 <unknown>
#7 0x605021762a73 <unknown>
#8 0x605021733c93 <unknown>
#9 0x60502173465e <unknown>
#10 0x6050219c008b <unknown>
#11 0x6050219c4005 <unknown>
#12 0x6050219ae491 <unknown>
#13 0x6050219c4b92 <unknown>
#14 0x6050219939ef <unknown>
#15 0x6050219eadf8 <unknown>
#16 0x6050219eafcb <unknown>
#17 0x6050219faae4 <unknown>
#18 0x73ec6ac94ac3 <unknown>
